In [ ]:
import d6tpipe

In [ ]:
d6tpipe.api.ConfigManager().init()

In [ ]:
api = d6tpipe.api.APIClient() # to connect to api cloud repo

api = d6tpipe.api.APILocal() # to connect to the local files

In [ ]:
api.register('username','email','password') # to register

api.login('username','password') # to login

In [ ]:
# this shows the list of pipes on cloud repo API
api.list_pipes()

In [ ]:
# connect to the data pipe
pipe = d6tpipe.Pipe(api,'intro-stat-learning')

In [ ]:
# preview files and size

pipe.pull_preview()

In [ ]:
# download all data in the path mentioned by pipe.dirpath in local computer

pipe.pull()

In [ ]:
# shows local files
pipe.dirpath

In [ ]:
# load local files 

import pandas as pd

df = pd.read_csv(pipe.dirpath/'Advertising.csv',**pipe.schema['pandas']) # use schema to quickly load data

In [ ]:
# read multiple files into dask

import dask.dataframe as dd

files = pipe.filepaths(include='Advertising*.csv')
ddf = dd.read_csv(files, **pipe.schema['dask'])

print(ddf.head())

In [ ]:
# open most recent csv

df = pd.read_csv(pipe.files(sortby='mod')[-1])

# save data to local files

df.to_csv(pipe.dirpath/'new_file.csv')

In [ ]:
# this will transfer all file in the given path to the local directory of d6tpipe

pipe.import_dir('../Loan Data/Filter_Data')

In [ ]:
# to view all the files at the local directory
pipe.scan_local()

In [ ]:
# to show local file repository
api.filerepo

### Manage Local File Storage

In [ ]:
# option1 : set custom path BEFORE init

# d6tpipe.api.ConfigManager().init({'filerepo':'/path/'})

# option2 : move file repo AFTER init

# api = d6tpipe.api.APILocal()
# api.move_repo('/new/path')
# print(api.filerepo)

# option3 : manually move file repo and update config

# d6tpipe.api.ConfigManager().update({'filerepo':'/path/'})

In [ ]:
# to create a new pipe in cloud repo API, this is also known as registering pipe

response, data = d6tpipe.upsert_pipe(api,{'name':'LoanData'})

In [ ]:
# pushing files from local to remote

pipe = d6tpipe.pipe.Pipe(api, 'LoanData')

files = ['../LoanData_2016.csv',
          '../LoanData_2017.csv',
        'C../LoanData_2018.csv']
pipe.push(files)

# pipe.push(include='*.csv', exclude='oldfile*.csv') # apply file filters

In [ ]:
# push the files in the default folder to remote cloud repo

# pipe.push_preview() # preview

# pipe.push() # execute


### Expired Token

If there is an error: "ClientError: An error occurred (ExpiredToken) when calling the PutObject operation: the provided token has expired". For security reasons we recieve short-term credentials.We can force renewal

In [ ]:
# pipe._reset_credentials()

### Sharing Remote Data Storage

In [ ]:
# give another user access

settings = {"username":"another-user","role":"read"} # role can be read, write or admin

d6tpipe.upsert_permissions(api, 'pipe-name', settings)

#make data repo public

settings = {"username":"public","role":"read"}

d6tpipe.upsert_permissions(api, 'remote-name', settings)

# view permission (owner only)

pipe.cnxpipe.permissions.get()